In [ ]:
#составление переводного глоссария

In [1]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import numpy as np  
stemmer = WordNetLemmatizer()
from string import punctuation
import numpy

In [2]:
punctuation = punctuation.replace('\"', "")

In [4]:
data = pd.read_csv('pi2.csv', sep=';', low_memory=False)
data = data[['string_id','Russian', 'English']]
index_names = data[data['Russian'].str.contains("тест" or "test")==True].index
# # index_names
data.drop(index_names, inplace = True)
data = data.dropna()
data = data.drop_duplicates()
data = data[data.Russian != data.English]
index_names2 = data[data['string_id'].str.contains("achieve", "title") ==True].index
data.drop(index_names2, inplace = True)
data = data[data['English'].apply(lambda x: len(x.split()) <= 3)]
data = data.drop_duplicates(subset=['English', 'Russian'], keep='last')
data = data[['Russian', 'English']]

In [6]:
import pickle
with open('keywords.pkl', 'rb') as f:
    keywords = pickle.load(f)

In [8]:
data_dict = dict(zip(data.English,data.Russian))

In [10]:
termbase = {}
for x in data_dict:
    if x.lower().strip(punctuation) in keywords:
        termbase[x.strip(punctuation)] = data_dict[x].strip(punctuation)

In [15]:
termbase_df = pd.DataFrame.from_dict(termbase, orient='index')
termbase_df.drop_duplicates()

,0
Accessory,Аксессуар
Achievement reward,Награда за достижение
Today,Cегодня
Construction,Строительство
Watch,Смотреть
...,...
stage,этап
come,прийти
rainbow,радуга
action,действие


In [12]:
with open('D:\\python\\CL\\alignment\\termbase_unigrams_X.pkl', 'rb') as f:
    unigrams = pickle.load(f)
with open('D:\\python\\CL\\alignment\\termbase_3_X.pkl', 'rb') as f:
    unigrams3 = pickle.load(f)

In [14]:
termbase_en_lower = [x.lower() for x in termbase]
for x in unigrams:
    if x not in termbase_en_lower:
        termbase[x] = unigrams[x]
print(termbase)         

{'Accessory': 'Аксессуар', 'Achievement reward': 'Награда за достижение', 'Today': 'Cегодня', 'Construction': 'Строительство', 'Watch': 'Смотреть', 'Video Ad': 'Реклама', 'Exciting News': 'Интересные новости', 'Refresh': 'Обновить', 'Alright': 'Хорошо', 'Accessory Slot': 'Ячейка для аксессуаров', 'Argh': 'Ар-р-р-х', 'Refill': 'Пополните запас', 'Unique offer': 'Уникальное предложение', 'Attention': 'Внимание', 'Rare Item': 'Редкий предмет', 'New Slot': 'Новая ячейка', 'Join': 'В бой', 'Tent': 'Палатка', 'Overgrown House': 'Заросший домик', 'Stranger': 'Незнакомец', 'Friendship level': 'Уровень дружелюбия', 'Fuel Supply': 'Запас топлива', 'Snowball Supply': 'Запас снега', 'Water Supply': 'Запас воды', 'Download content update': 'Загрузить обновление контента', 'Bank': 'Банк', 'Fail': 'Неудача', 'Infrastructure': 'Инфраструктура', 'Journal': 'Дневник', 'Balance': 'Сбалансированность', 'Paradise Island': 'Райский Остров', 'True Paradise': 'Настоящий Райский', 'Island': 'Остров', 'Generosi

In [22]:
#поиск лучших соответствий по топ-3 ближайшим словам
def options():
    print("Какой термин вы ищете?")
    term = input()
    with open('D:\\python\\CL\\alignment\\termbase_3_X.pkl', 'rb') as f:
        unigrams3 = pickle.load(f)
    print(unigrams3[term])

In [169]:
options()

Какой термин вы ищете?
unlock
['открыть', 'достижение', 'получить']


In [47]:
#проверим сколько у нас совпадений со сделанной вручную базой
df_tb = pd.read_csv('PI2TB.csv', sep=',')
df_tb = df_tb.English.drop_duplicates()
list_tb = df_tb.tolist()
list_tb_clean = [x.lower().strip() for x in list_tb]
len(list_tb_clean)

2427

In [52]:
our_tb_clean = [x.lower().strip() for x in list(termbase.keys())]
common = [x for x in list_tb_clean if x in our_tb_clean]
print(len(common), common)

701 ['construction', 'business', 'loading', 'video ad', 'treasure chest', 'water supply', 'snowball supply', 'cancel', 'fuel supply', 'accessory slot', 'demolish', 'stranger', 'electricity', 'energy', 'friendship level', 'crystal', 'staff', 'tourist', 'capacity', 'entertainment', 'bank', 'bonus', 'chapter', 'upgrade', 'journal', 'balance', 'generosity', 'luck', 'ranking', 'shop', 'difficulty', 'sunshine center', 'souvenir store', 'cabin', 'red pitcher cafe', 'gurgle attraction', 'splash attraction', 'bungalow', 'water slide', 'clown water slide', 'circus', 'phoenix hotel', 'irish pub', 'pranzo pizzeria', 'perfume store', 'windmill', 'pier', 'deluxe hotel', 'aroma coffee house', 'solar power plant', 'bavarian inn', 'mini hotel', 'mansion', 'dolphinarium', 'beach anticafe', 'recreation park', 'leafy gazebo', 'generator', 'aeronaut cafe', 'villa', 'renaissance hotel', 'chest', 'island hotel', 'wild west attraction', 'ice cream truck', 'cupid hotel', 'leprechaun hill hotel', 'golden barrel

In [16]:
termbase_df.to_csv("termbase1.csv", encoding = "utf-8")